# DEMO


The following notebook is a demonstration on how to use the tools in this repository to create
Initial data


In [57]:
import datasets
from data_preparation.synthetic_dataset_creation import create_or_load_inflector, create_or_load_stop_word_dictionary
from data_preparation.sentence_modifier import SentenceModifier
import spacy
from grammar_error_correction.grammar_error_correction import KEEP
import pandas as pd
from grammar_error_correction.grammar_error_correction import GrammarErrorCorrector
from transformers import AutoTokenizer

## Dataset structure

```json
[
  {
    "original_words": ["@@PADDING@@", "El", "ama", "los", "gatos", "."],
    "modified_words": ["@@PADDING@@", "El", "ama", "gatos", "."],
    "labels": ["$KEEP", "$KEEP", "$APPEND_los", "$KEEP", "$KEEP"]
  },
  
  {}
  
]
```

## LABELS

- KEEP: No change is needed in this position
- DELETE`: The word corresponding to this position has to be removed
- APPEND_<word>: append the token `<word>` after the position of this label 
- SPLIT_<place>: split the word at this position at the character `<place>`
- REPLACE_<word>: replace the word at this position for `<word>`

For the creation of our dataset we need a base dataset of correct sentences. In our case we picked https://huggingface.co/datasets/crscardellino/spanish_billion_words. For this small demo we are going to need just a few examples. About 10 will do

In [58]:
raw_dataset_path = 'spanish_billion_words'
sample_size = 10
texts = datasets.load_dataset(raw_dataset_path, trust_remote_code=True)['train'][:10]['text']
print(texts)

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

['Source Wikisource librodot com SENTIDO Y SENSIBILIDAD JANE AUSTEN CAPITULO I', 'La familia Dashwood llevaba largo tiempo afincada en Sussex', 'Su propiedad era de buen tamaño y en el centro de ella se encontraba la residencia Norland Park donde la manera tan digna en que habían vivido por muchas generaciones llegó a granjearles el respeto de todos los conocidos del lugar', 'El último dueño de esta propiedad había sido un hombre soltero que alcanzó una muy avanzada edad y que durante gran parte de su existencia tuvo en su hermana una fiel compañera y ama de casa', 'Pero la muerte de ella ocurrida diez años antes que la suya produjo grandes alteraciones en su hogar', 'Para compensar tal pérdida invitó y recibió en su casa a la familia de su sobrino el señor Henry Dashwood el legítimo heredero de la finca Norland y la persona a quien se proponía dejarla en su testamento', 'En compañía de su sobrino y sobrina y de los hijos de ambos la vida transcurrió confortablemente para el anciano ca

In [59]:
texts

['Source Wikisource librodot com SENTIDO Y SENSIBILIDAD JANE AUSTEN CAPITULO I',
 'La familia Dashwood llevaba largo tiempo afincada en Sussex',
 'Su propiedad era de buen tamaño y en el centro de ella se encontraba la residencia Norland Park donde la manera tan digna en que habían vivido por muchas generaciones llegó a granjearles el respeto de todos los conocidos del lugar',
 'El último dueño de esta propiedad había sido un hombre soltero que alcanzó una muy avanzada edad y que durante gran parte de su existencia tuvo en su hermana una fiel compañera y ama de casa',
 'Pero la muerte de ella ocurrida diez años antes que la suya produjo grandes alteraciones en su hogar',
 'Para compensar tal pérdida invitó y recibió en su casa a la familia de su sobrino el señor Henry Dashwood el legítimo heredero de la finca Norland y la persona a quien se proponía dejarla en su testamento',
 'En compañía de su sobrino y sobrina y de los hijos de ambos la vida transcurrió confortablemente para el anci

## Text transformations

Now that we haver our texts samples we can start creating our dataset. This will require us to create an `Inflector` and a `StopWordDictionary`

In [60]:
inflector_path = 'data/inflector.json'
stop_word_dictionary_path = 'data/stop_word_dictionary.json'
nlp_model = spacy.load('es_core_news_sm')

inflector = create_or_load_inflector(inflector_path, raw_dataset_path, nlp_model, sample_size)
stop_word_dictionary = create_or_load_stop_word_dictionary(stop_word_dictionary_path, raw_dataset_path, nlp_model,
                                                           sample_size)

In [61]:
inflector.inflections['NOUN']['conocido']

['conocido', 'conocidos']

In [62]:
stop_word_dictionary.stop_words_by_pos['DET']

['suya',
 'esta',
 'la',
 'La',
 'El',
 'todo',
 'una',
 'un',
 'los',
 'Su',
 'el',
 'sus',
 'todos',
 'su']

These two objects will be used to change the morphology of words and replace stop words within a POS respectively.

In [63]:
sentence_modifier = SentenceModifier(nlp_model, inflector, stop_word_dictionary, transformation_rate=0.5)

Using this sentence modifier we can start applying modifications to our texts

## Morphological changes

In [76]:
sentence = nlp_model(texts[7])
print(f'SENTENCE: {sentence}')
words = sentence_modifier.sentence_words(sentence)
tokens = [None] + list(sentence)
initial_labels = [KEEP] * len(words)

print('________ORIGINAL SENTENCE________')
print(pd.DataFrame({'words': words, 'tokens': tokens, 'labels': initial_labels}))
print()

changed_words, changed_tokens, labels = sentence_modifier.transform_morphology(words, tokens, initial_labels)

print('________MODIFIED SENTENCE________')
print(pd.DataFrame({'words': changed_words, 'tokens': changed_tokens, 'labels': labels}))

SENTENCE: Su apego a todos ellos fue creciendo con el tiempo
________ORIGINAL SENTENCE________
          words     tokens labels
0   @@PADDING@@       None  $KEEP
1            Su         Su  $KEEP
2         apego      apego  $KEEP
3             a          a  $KEEP
4         todos      todos  $KEEP
5         ellos      ellos  $KEEP
6           fue        fue  $KEEP
7     creciendo  creciendo  $KEEP
8           con        con  $KEEP
9            el         el  $KEEP
10       tiempo     tiempo  $KEEP

________MODIFIED SENTENCE________
          words     tokens       labels
0   @@PADDING@@       None        $KEEP
1            Su         Su        $KEEP
2         apego      apego        $KEEP
3             a          a        $KEEP
4         todos      todos        $KEEP
5         ellos      ellos        $KEEP
6           fue        fue        $KEEP
7     creciendo  creciendo        $KEEP
8           con        con        $KEEP
9           los       None  $REPLACE_el
10       tiempo     ti

## Replace a stop word

In [13]:
sentence = nlp_model(texts[1])
print(f'SENTENCE: {sentence}')
words = sentence_modifier.sentence_words(sentence)
tokens = [None] + list(sentence)
initial_labels = [KEEP] * len(words)

print('________ORIGINAL SENTENCE________')
print(pd.DataFrame({'words': words, 'tokens': tokens, 'labels': initial_labels}))
print()

changed_words, changed_tokens, labels = sentence_modifier.transform_within_poss(words, tokens, initial_labels)

print('________MODIFIED SENTENCE________')
print(pd.DataFrame({'words': changed_words, 'tokens': changed_tokens, 'labels': labels}))

SENTENCE: La familia Dashwood llevaba largo tiempo afincada en Sussex
________ORIGINAL SENTENCE________
         words    tokens labels
0  @@PADDING@@      None  $KEEP
1           La        La  $KEEP
2      familia   familia  $KEEP
3     Dashwood  Dashwood  $KEEP
4      llevaba   llevaba  $KEEP
5        largo     largo  $KEEP
6       tiempo    tiempo  $KEEP
7     afincada  afincada  $KEEP
8           en        en  $KEEP
9       Sussex    Sussex  $KEEP

________MODIFIED SENTENCE________
         words    tokens       labels
0  @@PADDING@@      None        $KEEP
1           La        La        $KEEP
2      familia   familia        $KEEP
3     Dashwood  Dashwood        $KEEP
4      llevaba   llevaba        $KEEP
5        largo     largo        $KEEP
6       tiempo    tiempo        $KEEP
7     afincada  afincada        $KEEP
8          con      None  $REPLACE_en
9       Sussex    Sussex        $KEEP


## Elimination of stop words

In [5]:
sentence = nlp_model(texts[1])
print(f'SENTENCE: {sentence}')
words = sentence_modifier.sentence_words(sentence)
tokens = [None] + list(sentence)
initial_labels = [KEEP] * len(words)

print('________ORIGINAL SENTENCE________')
print(pd.DataFrame({'words': words, 'tokens': tokens, 'labels': initial_labels}))
print()

changed_words, changed_tokens, labels = sentence_modifier.transform_elimination(words, tokens, initial_labels)

print('________MODIFIED SENTENCE________')
print(pd.DataFrame({'words': changed_words, 'tokens': changed_tokens, 'labels': labels}))

SENTENCE: La familia Dashwood llevaba largo tiempo afincada en Sussex
________ORIGINAL SENTENCE________
         words    tokens labels
0  @@PADDING@@      None  $KEEP
1           La        La  $KEEP
2      familia   familia  $KEEP
3     Dashwood  Dashwood  $KEEP
4      llevaba   llevaba  $KEEP
5        largo     largo  $KEEP
6       tiempo    tiempo  $KEEP
7     afincada  afincada  $KEEP
8           en        en  $KEEP
9       Sussex    Sussex  $KEEP

________MODIFIED SENTENCE________
         words    tokens      labels
0  @@PADDING@@      None       $KEEP
1           La        La       $KEEP
2      familia   familia       $KEEP
3     Dashwood  Dashwood       $KEEP
4      llevaba   llevaba       $KEEP
5        largo     largo       $KEEP
6       tiempo    tiempo       $KEEP
7     afincada  afincada  $APPEND_en
8       Sussex    Sussex       $KEEP


## Adding stop words

In [6]:
sentence = nlp_model(texts[1])
print(f'SENTENCE: {sentence}')
words = sentence_modifier.sentence_words(sentence)
tokens = [None] + list(sentence)
initial_labels = [KEEP] * len(words)

print('________ORIGINAL SENTENCE________')
print(pd.DataFrame({'words': words, 'tokens': tokens, 'labels': initial_labels}))
print()

changed_words, changed_tokens, labels = sentence_modifier.transform_adding(words, tokens, initial_labels)

print('________MODIFIED SENTENCE________')
print(pd.DataFrame({'words': changed_words, 'tokens': changed_tokens, 'labels': labels}))

SENTENCE: La familia Dashwood llevaba largo tiempo afincada en Sussex
________ORIGINAL SENTENCE________
         words    tokens labels
0  @@PADDING@@      None  $KEEP
1           La        La  $KEEP
2      familia   familia  $KEEP
3     Dashwood  Dashwood  $KEEP
4      llevaba   llevaba  $KEEP
5        largo     largo  $KEEP
6       tiempo    tiempo  $KEEP
7     afincada  afincada  $KEEP
8           en        en  $KEEP
9       Sussex    Sussex  $KEEP

________MODIFIED SENTENCE________
          words    tokens   labels
0   @@PADDING@@      None    $KEEP
1            La        La    $KEEP
2       familia   familia    $KEEP
3      Dashwood  Dashwood    $KEEP
4       llevaba   llevaba    $KEEP
5        estáis      None  $DELETE
6         largo     largo    $KEEP
7        tiempo    tiempo    $KEEP
8      afincada  afincada    $KEEP
9            en        en    $KEEP
10       Sussex    Sussex    $KEEP


## Token fusion

In [7]:
sentence = nlp_model(texts[1])
print(f'SENTENCE: {sentence}')
words = sentence_modifier.sentence_words(sentence)
tokens = [None] + list(sentence)
initial_labels = [KEEP] * len(words)

print('________ORIGINAL SENTENCE________')
print(pd.DataFrame({'words': words, 'tokens': tokens, 'labels': initial_labels}))
print()

changed_words, changed_tokens, labels = sentence_modifier.transform_token_fusion(words, tokens, initial_labels)

print('________MODIFIED SENTENCE________')
print(pd.DataFrame({'words': changed_words, 'tokens': changed_tokens, 'labels': labels}))

SENTENCE: La familia Dashwood llevaba largo tiempo afincada en Sussex
________ORIGINAL SENTENCE________
         words    tokens labels
0  @@PADDING@@      None  $KEEP
1           La        La  $KEEP
2      familia   familia  $KEEP
3     Dashwood  Dashwood  $KEEP
4      llevaba   llevaba  $KEEP
5        largo     largo  $KEEP
6       tiempo    tiempo  $KEEP
7     afincada  afincada  $KEEP
8           en        en  $KEEP
9       Sussex    Sussex  $KEEP

________MODIFIED SENTENCE________
             words    tokens    labels
0      @@PADDING@@      None     $KEEP
1               La        La     $KEEP
2          familia   familia     $KEEP
3  Dashwoodllevaba      None  $SPLIT_8
4            largo     largo     $KEEP
5           tiempo    tiempo     $KEEP
6         afincada  afincada     $KEEP
7               en        en     $KEEP
8           Sussex    Sussex     $KEEP


## Putting it all together

In [77]:
sentence = nlp_model(texts[1])
print(f'SENTENCE: {sentence}')

changed_words, changed_tokens, labels = sentence_modifier.randomly_transform(sentence)

print('________MODIFIED SENTENCE________')
print(pd.DataFrame({'words': changed_words, 'tokens': changed_tokens, 'labels': labels}))

SENTENCE: La familia Dashwood llevaba largo tiempo afincada en Sussex
________MODIFIED SENTENCE________
             words    tokens       labels
0      @@PADDING@@      None   $APPEND_La
1          familia   familia        $KEEP
2  Dashwoodllevaba      None     $SPLIT_8
3            largo     largo        $KEEP
4           tiempo    tiempo        $KEEP
5           siente      None      $DELETE
6         afincada  afincada        $KEEP
7               De      None  $REPLACE_en
8           Sussex    Sussex        $KEEP


## Grammar Error Correction

Now that we have our text with errors we can correct it using the labels

In [78]:
gec_no_model = GrammarErrorCorrector(None, None)

print('________MODIFIED SENTENCE________')
print(pd.DataFrame({'words': changed_words, 'labels': labels}))

corrected_words, corrected_labels = gec_no_model.correct_label_errors(changed_words, labels)

print('________CORRECTED SENTENCE________')
print(pd.DataFrame({'words': corrected_words, 'labels': corrected_labels}))

________MODIFIED SENTENCE________
             words       labels
0      @@PADDING@@   $APPEND_La
1          familia        $KEEP
2  Dashwoodllevaba     $SPLIT_8
3            largo        $KEEP
4           tiempo        $KEEP
5           siente      $DELETE
6         afincada        $KEEP
7               De  $REPLACE_en
8           Sussex        $KEEP
________CORRECTED SENTENCE________
         words labels
0  @@PADDING@@  $KEEP
1           La  $KEEP
2      familia  $KEEP
3     Dashwood  $KEEP
4      llevaba  $KEEP
5        largo  $KEEP
6       tiempo  $KEEP
7     afincada  $KEEP
8           en  $KEEP
9       Sussex  $KEEP


## Preparing the data for training

Before we can use this data for training we have to tackle a particular problem. While our dataset is created based on word-wise tokenization BERT and similar models use word-piece tokenization. 

In [79]:
model_id = "MMG/xlm-roberta-large-ner-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_tokens('@@PADDING@@')
print(f'SENTENCE: {texts[1]}')
print(f'SPACY TOKENS: {list(nlp_model.tokenizer(texts[1]))}')
print(f'ROBERTA TOKENS: {tokenizer.tokenize(texts[1])}')

/home/gustavo/Documents/mlt-project/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SENTENCE: La familia Dashwood llevaba largo tiempo afincada en Sussex
SPACY TOKENS: [La, familia, Dashwood, llevaba, largo, tiempo, afincada, en, Sussex]
ROBERTA TOKENS: ['▁La', '▁familia', '▁Das', 'h', 'wood', '▁lleva', 'ba', '▁largo', '▁tiempo', '▁afin', 'cada', '▁en', '▁Sus', 'sex']


## Token-Label alignment

This means that we need to align our dataset labels with the actual tokens expected by the model

In [80]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["modified_words"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [81]:
example = {
    'modified_words': [changed_words],
    'ner_tags': [labels]
}

aligned_data = tokenize_and_align_labels(examples=example)
print(aligned_data)
aligned_data = {k: v[0] for k, v in aligned_data.items()}
aligned_data['token_text'] = tokenizer.convert_ids_to_tokens(aligned_data['input_ids'])
print(pd.DataFrame(aligned_data))

{'input_ids': [[0, 250002, 8650, 1858, 127, 25876, 1229, 120550, 23321, 7493, 167264, 17770, 23315, 262, 26832, 13802, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, '$APPEND_La', '$KEEP', '$SPLIT_8', -100, -100, -100, -100, '$KEEP', '$KEEP', '$DELETE', '$KEEP', -100, '$REPLACE_en', '$KEEP', -100, -100]]}
    input_ids  attention_mask       labels   token_text
0           0               1         -100          <s>
1      250002               1   $APPEND_La  @@PADDING@@
2        8650               1        $KEEP     ▁familia
3        1858               1     $SPLIT_8         ▁Das
4         127               1         -100            h
5       25876               1         -100         wood
6        1229               1         -100          lle
7      120550               1         -100         vaba
8       23321               1        $KEEP       ▁largo
9        7493               1        $KEEP      ▁tiempo
10     167264               